In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torchvision

from PIL import Image
import os

In [2]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [13]:
# path
input_path = "/home/piai/바탕화면/hanja_crop_50/new50/output_crop/"

In [14]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
    'validation':
    transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        normalize
    ]),
}

image_datasets = {
    'train': 
    datasets.ImageFolder(input_path + 'train', data_transforms['train']),
    'validation': 
    datasets.ImageFolder(input_path + 'validation', data_transforms['validation'])
}

dataloaders = {
    'train':
    torch.utils.data.DataLoader(image_datasets['train'],
                                batch_size=32,
                                shuffle=True,
                                num_workers=0),  # for Kaggle
    'validation':
    torch.utils.data.DataLoader(image_datasets['validation'],
                                batch_size=32,
                                shuffle=False,
                                num_workers=0)  # for Kaggle
}

In [15]:
dataiter = iter(dataloaders['train'])
images, labels = dataiter.next()
print(labels)

tensor([3218, 2263, 2199,  559, 1333, 1094, 2767, 1756, 3695, 3366, 2582, 3956,
        4004, 2703, 3583, 3124, 2462, 1181, 1211,  529, 4508, 4740,  979, 3630,
        2799, 3890,  585, 3092,  707, 2816, 2616, 2785])


In [18]:
classes = image_datasets['train'].classes
#classes

In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [21]:
model = models.resnet101(pretrained=True).to(device)
    
for param in model.parameters():
    param.requires_grad = False   
    
model.fc = nn.Sequential(
    nn.Linear(2048, 128),
    nn.ReLU(inplace=True),
    nn.Linear(128, 4893),
    #nn.LogSoftmax(dim=1) ############ 추가
    #nn.Linear(128,1422)에서 1422는 class 개수로 맞추기
).to(device)

In [22]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters())

In [23]:
import time
import datetime

In [24]:
def train_model(model, criterion, optimizer, num_epochs=3):
    for epoch in range(num_epochs):
        start = time.time()  # 시작 시간 저장
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                #return inputs, outputs, labels
                loss = criterion(outputs, labels)
                
                if phase == 'train':
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                        epoch_loss,
                                                        epoch_acc))
            
        torch.save(model.state_dict(), '/home/piai/바탕화면/hanja_crop_50/new50/weights/weights_{}.h5'.format(epoch))
        sec = time.time()-start
        times = str(datetime.timedelta(seconds=sec)).split(".")
        times = times[0]
        print(times)
        print()
    return model

In [ ]:
model_trained = train_model(model, criterion, optimizer, num_epochs=30)

Epoch 1/30
----------
train loss: 8.5099, acc: 0.0001
validation loss: 8.4987, acc: 0.0002
0:17:18

Epoch 2/30
----------
train loss: 8.5035, acc: 0.0001
validation loss: 8.4968, acc: 0.0002
0:17:15

Epoch 3/30
----------
train loss: 8.5023, acc: 0.0001
validation loss: 8.4961, acc: 0.0002
0:17:12

Epoch 4/30
----------
train loss: 8.5018, acc: 0.0001
validation loss: 8.4958, acc: 0.0002
0:17:13

Epoch 5/30
----------
train loss: 8.5016, acc: 0.0001
validation loss: 8.4957, acc: 0.0002
0:17:13

Epoch 6/30
----------
train loss: 8.5015, acc: 0.0001
validation loss: 8.4957, acc: 0.0002
0:17:12

Epoch 7/30
----------
train loss: 8.5015, acc: 0.0001
validation loss: 8.4957, acc: 0.0002
0:17:33

Epoch 8/30
----------
train loss: 8.5015, acc: 0.0001
validation loss: 8.4957, acc: 0.0002
0:17:37

Epoch 9/30
----------
train loss: 8.5015, acc: 0.0001
validation loss: 8.4956, acc: 0.0002
0:18:05

Epoch 10/30
----------
train loss: 8.5015, acc: 0.0001
validation loss: 8.4956, acc: 0.0002
0:17:31
